In [1]:
import numpy as np
import pandas as pd
import json, os, sys

In [2]:
bert_results = pd.read_json('results/bert_results_25august.json')
roberta_results = pd.read_json('results/roberta_results_25august.json')
t5_results = pd.read_json('results/t5_results_25august.json')

# NOTE

The results as-is from Colab have some issues due to the way I have constructed them. 

- Please CTRL-F and look for `"]["`. Replace with `"],["`
- Please insert an opening square bracket at start, and a closing one at the end. (enclose the whole contents into square brackets.)

These two steps fix the issues and can be read with `pd.read_json()`

In [3]:
final_dfs = []
for df in [bert_results, roberta_results]:
    all_vals = []
    for _, row in df.stack().to_frame().droplevel(1).iterrows():
        values = row.values
        for val in values:
            for item in val:
                if val is not None:
                    all_vals.append(item)
    final_dfs.append(all_vals)

for df in [t5_results]:
    all_vals = []
    for _, row in df.stack().to_frame().droplevel(1).iterrows():
        values = row.values
        all_vals.append(values[0])
    final_dfs.append(all_vals)

In [4]:
bert_results, roberta_results, t5_results = pd.DataFrame(
    final_dfs[0]), pd.DataFrame(final_dfs[1]), pd.DataFrame(final_dfs[2])

bert_results.to_json('results/bert_results_25august_clean.json')
roberta_results.to_json('results/roberta_results_25august_clean.json')
t5_results.to_json('results/t5_results_25august_clean.json')

In [74]:
# convert answer token to lowercase as we used uncased version of models
# compute 1/0 if answer is correct or not
for df in [bert_results, roberta_results, t5_results]:
    df['processed_answer'] = df['answer'].apply(lambda x: str(x).lower())
    df['processed_pred'] = df['token_str'].apply(lambda x: str(x).lower().strip())
    df['correct'] = np.where(df['processed_answer'] == df['processed_pred'], 1, 0)

In [75]:
t5_results.groupby('ids').first()

,sequence,token_str,answer,processed_answer,correct,processed_pred
ids,,,,,,
0001b29ee9e518b252beaedd07f7193c4c5bd405,Total UK exports of services (excluding travel...,17.0%,14.3%,14.3%,0,17.0%
000aba0de1890391d98fbba20af8abe629a95452,"Between 9 March and 30 June 2020, prior to the...",61,"5,330","5,330",0,none
000cbe156681cb91437a5f0953c0047f83e47f35,"In January to February 2020, 26% of adults in ...",26%,18%,18%,0,26%
00139eecad6df8931ffea624f73a530b0a1acb78,"“While they were more optimistic, young people...",many,42%,42%,0,many
005aed79078fc309e9d95f917bcb9d15dbff6d76,5% of children living in a city reported “ofte...,5%,19.5%,19.5%,0,5%
...,...,...,...,...,...,...
ffdef2a0629d19756210d13e448248ba983eb355,38% of adults said they had run out of money t...,38% of adults saying they were,12%,12%,0,38% of adults saying they were
ffe1db00c7b8cea213ab00eb0f57d28f1cb6dcc0,"Similar to ,, Ischaemic heart disease was the ...",",",previous years,previous years,0,","
ffec045e37d60b9eea6efead436c1d5edfad275a,Total current healthcare expenditure in the UK...,2018,2018,2018,1,2018


In [89]:
t5_results.groupby('ids').first().correct.sum()

600

In [77]:
t5_results[t5_results['ids'] == 'ffec045e37d60b9eea6efead436c1d5edfad275a']

,sequence,token_str,answer,ids,processed_answer,correct,processed_pred
20100,Total current healthcare expenditure in the UK...,2018,2018,ffec045e37d60b9eea6efead436c1d5edfad275a,2018,1,2018
20101,Total current healthcare expenditure in the UK...,2018,2018,ffec045e37d60b9eea6efead436c1d5edfad275a,2018,1,2018
20102,Total current healthcare expenditure in the UK...,2018.,2018,ffec045e37d60b9eea6efead436c1d5edfad275a,2018,0,2018.
20103,Total current healthcare expenditure in the UK...,2018,2018,ffec045e37d60b9eea6efead436c1d5edfad275a,2018,1,2018
20104,Total current healthcare expenditure in the UK...,2018,2018,ffec045e37d60b9eea6efead436c1d5edfad275a,2018,1,2018


In [78]:
t5_results[t5_results['ids'] == '0001b29ee9e518b252beaedd07f7193c4c5bd405']

,sequence,token_str,answer,ids,processed_answer,correct,processed_pred
1495,Total UK exports of services (excluding travel...,17.0%,14.3%,0001b29ee9e518b252beaedd07f7193c4c5bd405,14.3%,0,17.0%
1496,Total UK exports of services (excluding travel...,18.0%,14.3%,0001b29ee9e518b252beaedd07f7193c4c5bd405,14.3%,0,18.0%
1497,Total UK exports of services (excluding travel...,17.0%,14.3%,0001b29ee9e518b252beaedd07f7193c4c5bd405,14.3%,0,17.0%
1498,Total UK exports of services (excluding travel...,17.0%,14.3%,0001b29ee9e518b252beaedd07f7193c4c5bd405,14.3%,0,17.0%
1499,Total UK exports of services (excluding travel...,19.0%,14.3%,0001b29ee9e518b252beaedd07f7193c4c5bd405,14.3%,0,19.0%


In [99]:
bert_first_correct = bert_results.groupby('ids').first().correct.sum()
roberta_first_correct = roberta_results.groupby('ids').first().correct.sum()
t5_first_correct = t5_results.groupby('ids').first().correct.sum()

print(
    f"Bert results, first prediction correct: {bert_first_correct} out of 6070; ({bert_first_correct / 60.70})")
print(
    f"RoBERTa results, first prediction correct: {roberta_first_correct} out of 6070; ({roberta_first_correct/60.70}) ")
print(
    f"T5 results, first prediction correct: {t5_first_correct} out of 6070; ({t5_first_correct/60.70})")

Bert results, first prediction correct: 507 out of 6070; (8.352553542009884)
RoBERTa results, first prediction correct: 679 out of 6070; (11.186161449752882) 
T5 results, first prediction correct: 600 out of 6070; (9.884678747940692)


In [104]:
bert_any_correct = bert_results.groupby('ids').any().correct.sum()
roberta_any_correct = roberta_results.groupby('ids').any().correct.sum()
t5_any_correct = t5_results.groupby('ids').any().correct.sum()

print(
    f"Bert results, any prediction correct: {bert_any_correct} out of 6070; ({bert_any_correct / 60.70})")
print(
    f"RoBERTa results, any prediction correct: {roberta_any_correct} out of 6070; ({roberta_any_correct/60.70}) ")
print(
    f"T5 results, any prediction correct: {t5_any_correct} out of 6070; ({t5_any_correct/60.70})")

Bert results, any prediction correct: 795 out of 6070; (13.097199341021415)
RoBERTa results, any prediction correct: 1052 out of 6070; (17.331136738056014) 
T5 results, any prediction correct: 891 out of 6070; (14.678747940691927)
